# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from os import listdir, fspath
from os.path import isfile, join
import numpy_financial as npf
from datetime import date
from collections import OrderedDict
from dateutil.relativedelta import *

print (f"Python version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Numpy Financial Version {npf.__version__}")

Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Pandas Version: 1.1.3
Numpy version: 1.19.2
Numpy Financial Version 1.0.0


# Input Data

In [2]:
def calc_r(postedrate):
    """Calculates the effective annual rate for a Canadian Posted Rate based on Canada's legal Semi Annual Compounding
    and 12 monthly payments
    """
    semiannualrate = postedrate/2
    CanAnnualRate=((1+semiannualrate)**2-1)
    reff_monthly=(1+CanAnnualRate)**(1/12)-1  # Only deal in monthly calcs for this workbook
    r = reff_monthly*12 # back calculate an effective yearly rate based on the semi annual compounding
    return r

In [3]:
P = 100000 # principal
postedrate = 0.06 # annual interest rate - as quoted
r = calc_r(postedrate)
t = 25 # amortization period, years
n = 12 # number of payments per year - DO NOT CHANGE
startdate = (date(2021,1,1))
Addl_Principal = 200 # Additional Principal per period

#### Change Rate for Canada - Semi Annual Compounding
From https://vindeep.com/Corporate/InterestRateConversion.aspx and http://www.yorku.ca/amarshal/mortgage.htm   
(In second reference, note error in 2nd line of page, where it should be rM = (1.0609)^1/12-1)

# Basic Calculations

In [4]:
r_n = r/n  # monthly interest rate
totpymts = n*t # total number of payments

In [5]:
pymt = npf.pmt(r_n,totpymts,P)
per = 1
ipmt = npf.ipmt(r_n,per,totpymts,P)
ppmt = npf.ppmt(r_n,per,totpymts,P)
print(f"Principal of ${P}, {n} payments/year, {postedrate*100}% interest as posted, {t} years amortization, payments are ${-pymt:.2f} per period.")
print(f"For period {per}, the interest payment is ${-ipmt:.2f} and the principal payment is ${-ppmt:.2f}, not counting for additional payments per period of ${Addl_Principal:.2f}")

Principal of $100000, 12 payments/year, 6.0% interest as posted, 25 years amortization, payments are $639.81 per period.
For period 1, the interest payment is $493.86 and the principal payment is $145.94, not counting for additional payments per period of $200.00


# Build Amortization Table
From https://pbpython.com/amortization-model-revised.html

### Create generator

In [6]:
def amortize(principal, interest_rate, years, pmt, addl_principal=0, annual_payments=12, start_date=date.today()):
    """Generator to calculate the Amortization table
    This can be used to generate the dataframe
    
    Inputs are Principal amount, yearly interest rate
    years of amortization, Additional Principal per period, Annual number of payments
    and start date
    
    Output is Ordered Dictionary of results
    """
        
    # initialize the variables to keep track of the periods and running balances
    p = 1
    beg_balance = principal
    end_balance = principal

    while end_balance > 0:

        # Recalculate the interest based on the current balance
        interest = round(((interest_rate/annual_payments) * beg_balance), 2)

        # Determine payment based on whether or not this period will pay off the loan
        pmt = min(pmt, beg_balance + interest)
        principal = pmt - interest

        # Ensure additional payment gets adjusted if the loan is being paid off
        addl_principal = min(addl_principal, beg_balance - principal)
        end_balance = beg_balance - (principal + addl_principal)

        yield OrderedDict([('Month',start_date),
                           ('Period', p),
                           ('BeginBal', beg_balance),
                           ('Payment', pmt),
                           ('Principal', principal),
                           ('Interest', interest),
                           ('AddtnlPayment', addl_principal),
                           ('EndBal', end_balance)])

        # Increment the counter, balance and date
        p += 1
        start_date += relativedelta(months=1)
        beg_balance = end_balance

### Create Amortization Table

In [7]:
def AmortTable(principal, interest_rate, years,
                       addl_principal=0, annual_payments=12, start_date=date.today()):
    """
    Calculate the amortization schedule given the loan details as well as summary stats for the loan

    :param principal: Amount borrowed
    :param interest_rate: The annual interest rate for this loan
    :param years: Number of years for the loan
    
    :param annual_payments (optional): Number of payments in a year. DEfault 12.
    :param addl_principal (optional): Additional payments to be made each period. Default 0.
    :param start_date (optional): Start date. Default first of next month if none provided

    :return: 
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    
    # Payment stays constant based on the original terms of the loan
    payment = -round(npf.pmt(interest_rate/annual_payments, years*annual_payments, principal), 2)
    
    # Generate the schedule and order the resulting columns for convenience
    schedule = pd.DataFrame(amortize(principal, interest_rate, years, payment,
                                     addl_principal, annual_payments, start_date))
    schedule = schedule[["Period", "Month", "BeginBal", "Payment", "Principal", 
                          "Interest", "AddtnlPayment", "EndBal"]]
    
    # Convert to a datetime object to make subsequent calcs easier
    schedule["Month"] = pd.to_datetime(schedule["Month"])
    
    #Create a summary statistics table
    payoff_date = schedule["Month"].iloc[-1]
    posted_rt = 2*((1+interest_rate/12)**6-1)
    stats = pd.Series([payoff_date, schedule["Period"].count(), round(interest_rate*100,2),
                       round(posted_rt*100,2),years, principal, payment, addl_principal,
                       schedule["Interest"].sum()],
                       index=["Payoff Date", "Num Payments", "Effective Annual Interest Rate, %", 
                              "Posted Annual Rate, %","Amortization Period, Years", "Principal",
                             "Payment", "Additional Payment", "Total Interest"])
    
    return schedule, stats

### Get Results

In [8]:
sch,stats = AmortTable(principal=P,interest_rate=r,years=t,addl_principal=Addl_Principal,annual_payments=n,start_date=startdate)

sch.head()
#schedule.set_index('Period',inplace=True)

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
0,1,2021-01-01,100000.00,639.81,145.95,493.86,200.0,99654.05
1,2,2021-02-01,99654.05,639.81,147.66,492.15,200.0,99306.39
2,3,2021-03-01,99306.39,639.81,149.37,490.44,200.0,98957.02
3,4,2021-04-01,98957.02,639.81,151.10,488.71,200.0,98605.92
4,5,2021-05-01,98605.92,639.81,152.83,486.98,200.0,98253.09


In [9]:
stats

Payoff Date                          2036-01-01 00:00:00
Num Payments                                         181
Effective Annual Interest Rate, %                   5.93
Posted Annual Rate, %                                  6
Amortization Period, Years                            25
Principal                                         100000
Payment                                           639.81
Additional Payment                                   200
Total Interest                                   51186.9
dtype: object

# Analysis

## 1. Reproduce Original Table to Check

In [21]:
P1 = 469300 # principal
postedrate1 = 0.0299 # annual interest rate - as quoted
r1 = calc_r(postedrate1)
t1 = 25 # amortization period, years
n1 = 12 # number of payments per year - DO NOT CHANGE
startdate1 = (date(2019,6,21))
Addl_Principal1 = 0 # Additional Principal per period

In [22]:
sch1,stats1 = AmortTable(principal=P1,interest_rate=r1,years=t1,addl_principal=Addl_Principal1,annual_payments=n1,start_date=startdate1)

In [23]:
sch1.head()

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
0,1,2019-06-21,469300.00,2218.53,1056.41,1162.12,0,468243.59
1,2,2019-07-21,468243.59,2218.53,1059.02,1159.51,0,467184.57
2,3,2019-08-21,467184.57,2218.53,1061.65,1156.88,0,466122.92
3,4,2019-09-21,466122.92,2218.53,1064.28,1154.25,0,465058.64
4,5,2019-10-21,465058.64,2218.53,1066.91,1151.62,0,463991.73


In [24]:
sch1[33:36]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
33,34,2022-03-21,433021.27,2218.53,1146.25,1072.28,0,431875.02
34,35,2022-04-21,431875.02,2218.53,1149.08,1069.45,0,430725.94
35,36,2022-05-21,430725.94,2218.53,1151.93,1066.60,0,429574.01


In [39]:
stats1

Payoff Date                          2044-06-21 00:00:00
Num Payments                                         301
Effective Annual Interest Rate, %                   2.97
Posted Annual Rate, %                               2.99
Amortization Period, Years                            25
Principal                                         469300
Payment                                          2218.53
Additional Payment                                     0
Total Interest                                    196261
dtype: object

Within a few cents of the correct value at the end of the mortgage term per the spreadsheet (May 21 2022 - $429,573.65)

In [26]:
sch1[19:20]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
19,20,2021-01-21,448774.56,2218.53,1107.24,1111.29,0,447667.32


In [27]:
NewPrincipal = sch1.iloc[19]['EndBal']
print(NewPrincipal)

447667.32


## 2. Refinance at the New Rate

In [28]:
P2 = NewPrincipal # principal
postedrate2 = 0.0174 # annual interest rate - as quoted
r2 = calc_r(postedrate2)
t2 = 25 # amortization period, years
n2 = 12 # number of payments per year - DO NOT CHANGE
startdate2 = (date(2021,2,21))
Addl_Principal2 = 0 # Additional Principal per period

In [29]:
sch2,stats2 = AmortTable(principal=P2,interest_rate=r2,years=t2,addl_principal=Addl_Principal2,annual_payments=n2,start_date=startdate2)

In [33]:
sch2[11:12]

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
11,12,2022-01-21,434447.1,1839.96,1212.28,627.68,0,433234.82


In [31]:
stats2

Payoff Date                          2046-02-21 00:00:00
Num Payments                                         301
Effective Annual Interest Rate, %                   1.73
Posted Annual Rate, %                               1.74
Amortization Period, Years                            25
Principal                                         447667
Payment                                          1839.96
Additional Payment                                     0
Total Interest                                    104321
dtype: object

### 2A. Yearly payments of 15% of principal spread over each month of year

In [44]:
Addl_Principal2 = 0.15*P2/12 # additional amount per month
print(Addl_Principal2)

5595.8414999999995


In [45]:
sch2a,stats2a = AmortTable(principal=P2,interest_rate=r2,years=t2,addl_principal=Addl_Principal2,annual_payments=n2,start_date=startdate2)

In [46]:
sch2a.head()

,Period,Month,BeginBal,Payment,Principal,Interest,AddtnlPayment,EndBal
0,1,2021-02-21,447667.3200,1839.96,1193.18,646.78,5595.8415,440878.2985
1,2,2021-03-21,440878.2985,1839.96,1202.99,636.97,5595.8415,434079.4670
2,3,2021-04-21,434079.4670,1839.96,1212.81,627.15,5595.8415,427270.8155
3,4,2021-05-21,427270.8155,1839.96,1222.65,617.31,5595.8415,420452.3240
4,5,2021-06-21,420452.3240,1839.96,1232.50,607.46,5595.8415,413623.9825


In [47]:
stats2a

Payoff Date                          2026-05-21 00:00:00
Num Payments                                          64
Effective Annual Interest Rate, %                   1.73
Posted Annual Rate, %                               1.74
Amortization Period, Years                            25
Principal                                         447667
Payment                                          1839.96
Additional Payment                               5595.84
Total Interest                                     21016
dtype: object